In [1]:
import tensorflow as tf
import os
import glob
import time

print(f"Đang sử dụng TensorFlow phiên bản: {tf.__version__}")

Đang sử dụng TensorFlow phiên bản: 2.13.0


In [10]:
# =============================================================================
# ĐỊNH NGHĨA CÁC CLASS CUSTOM
# =============================================================================

@tf.keras.saving.register_keras_serializable(package="Custom")
class FinalModelCNN(tf.keras.Model):
    def __init__(self, input_shape_config, num_classes_config, **kwargs):
        super(FinalModelCNN, self).__init__(**kwargs)
        self.input_shape_config = input_shape_config
        self.num_classes_config = num_classes_config

        # 1. Base Model (CNN)
        self.base_model = tf.keras.applications.EfficientNetV2B2(
        weights=None,  # <-- ĐÃ SỬA
        include_top=False, 
        input_shape=self.input_shape_config
    )
        
        # 2. Lớp Pooling để giảm chiều dữ liệu
        self.gap = tf.keras.layers.GlobalAveragePooling2D(name="global_avg_pool")
        
        # 3. Các lớp Dense (Head)
        self.dense1 = tf.keras.layers.Dense(512, use_bias=False, kernel_regularizer=tf.keras.regularizers.l2(1e-5), name="dense_layer_1")
        self.bn1 = tf.keras.layers.BatchNormalization(name="batch_norm_1")
        self.act1 = tf.keras.layers.Activation('relu', name="activation_1")
        self.dropout1 = tf.keras.layers.Dropout(0.3, name="dropout_layer_1")
        
        self.dense2 = tf.keras.layers.Dense(256, use_bias=False, kernel_regularizer=tf.keras.regularizers.l2(1e-5), name="dense_layer_2")
        self.bn2 = tf.keras.layers.BatchNormalization(name="batch_norm_2")
        self.act2 = tf.keras.layers.Activation('relu', name="activation_2")
        self.dropout2 = tf.keras.layers.Dropout(0.2, name="dropout_layer_2")
        
        # 4. Lớp Output
        self.dense_output = tf.keras.layers.Dense(self.num_classes_config, activation='linear', dtype='float32', name="output_layer")

    def call(self, inputs, training=None):
        x = self.base_model(inputs, training=training)
        x = self.gap(x, training=training) 
        
        x = self.dense1(x)
        x = self.bn1(x, training=training); x = self.act1(x); x = self.dropout1(x, training=training)
        x = self.dense2(x)
        x = self.bn2(x, training=training); x = self.act2(x); x = self.dropout2(x, training=training)
        outputs = self.dense_output(x)
        return outputs

    def get_config(self):
        config = super(FinalModelCNN, self).get_config()
        config.update({
            'input_shape_config': self.input_shape_config,
            'num_classes_config': self.num_classes_config,
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

@tf.keras.saving.register_keras_serializable(package="Custom")
class MacroF1Score(tf.keras.metrics.Metric):
    def __init__(self, num_classes, name='f1_macro', **kwargs):
        super(MacroF1Score, self).__init__(name=name, **kwargs)
        self.num_classes = num_classes
        self.true_positives = self.add_weight(name='tp', shape=(num_classes,), initializer='zeros')
        self.false_positives = self.add_weight(name='fp', shape=(num_classes,), initializer='zeros')
        self.false_negatives = self.add_weight(name='fn', shape=(num_classes,), initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred_labels = tf.argmax(tf.nn.softmax(y_pred), axis=1)
        y_true_labels = tf.argmax(y_true, axis=1)
        cm = tf.math.confusion_matrix(y_true_labels, y_pred_labels, num_classes=self.num_classes, dtype=tf.float32)
        tp = tf.linalg.diag_part(cm)
        fp = tf.reduce_sum(cm, axis=0) - tp
        fn = tf.reduce_sum(cm, axis=1) - tp
        self.true_positives.assign_add(tp)
        self.false_positives.assign_add(fp)
        self.false_negatives.assign_add(fn)

    def result(self):
        precision = self.true_positives / (self.true_positives + self.false_positives + tf.keras.backend.epsilon())
        recall = self.true_positives / (self.true_positives + self.false_negatives + tf.keras.backend.epsilon())
        f1 = 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())
        macro_f1 = tf.reduce_mean(f1)
        return macro_f1

    def reset_state(self):
        self.true_positives.assign(tf.zeros(self.num_classes))
        self.false_positives.assign(tf.zeros(self.num_classes))
        self.false_negatives.assign(tf.zeros(self.num_classes))

    def get_config(self):
        config = super(MacroF1Score, self).get_config()
        config.update({'num_classes': self.num_classes})
        return config

print("Đã định nghĩa xong 2 class custom: 'FinalModelCNN' và 'MacroF1Score'")

Đã định nghĩa xong 2 class custom: 'FinalModelCNN' và 'MacroF1Score'


In [ ]:
# --- CẤU HÌNH ---
BASE_DIR = os.getcwd() 
SOURCE_MODEL_DIR = os.path.join(BASE_DIR, "models") 
DEST_TFLITE_DIR = os.path.join(BASE_DIR, "models_tflite") 
MODEL_TEST_NAME = "CNN_from_NPY_no_val.keras"
# ------------------

model_path = os.path.join(SOURCE_MODEL_DIR, MODEL_TEST_NAME)
tflite_name = MODEL_TEST_NAME.replace(".keras", ".tflite")
dest_path = os.path.join(DEST_TFLITE_DIR, tflite_name)

dest_path = os.path.normpath(dest_path) 
os.makedirs(os.path.normpath(DEST_TFLITE_DIR), exist_ok=True)

print(f"Thư mục làm việc hiện tại (BASE_DIR): {BASE_DIR}")
print(f"Đang tìm model Keras tại: {model_path}")
print(f"Sẽ lưu TFLite tại: {dest_path}")

if not os.path.exists(model_path):
    print(f"\n!!! LỖI KIỂM TRA: Không tìm thấy file tại đường dẫn trên. !!!")
else:
    print("\nKiểm tra: Đã tìm thấy file model. Bắt đầu chuyển đổi...")
    try:
        # =============================================================================
        # === SỬA LỖI: Cung cấp custom_objects TRỰC TIẾP ===
        # =============================================================================

        # 1. Tải toàn bộ model
        print(f"\n[1/3] Đang tải toàn bộ model (load_model) từ: {model_path}")
        
        # === PHẦN SỬA LỖI ===
        # (Đảm bảo bạn đã chạy Ô 2 để FinalModelCNN và MacroF1Score tồn tại)
        # Chúng ta tạo một từ điển (dict) để ánh xạ TÊN (string) sang CLASS (object)
        custom_objects = {
            "FinalModelCNN": FinalModelCNN,
            "MacroF1Score": MacroF1Score
        }

        # Truyền custom_objects VÀ compile=False vào hàm load_model
        model = tf.keras.models.load_model(
            model_path, 
            custom_objects=custom_objects,  # <--- SỬA LỖI Ở ĐÂY
            compile=False                   # <--- Vẫn giữ compile=False
        )
        # ==================================
        
        print("[1/3] Tải model thành công.")
        
        # 2. Khởi tạo bộ chuyển đổi
        print("[2/3] Đang khởi tạo bộ chuyển đổi TFLite...")
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        print("[2/3] Khởi tạo thành công.")
        
        # 3. Chuyển đổi và Lưu
        print("[3/3] Đang tiến hành lượng tử hóa và chuyển đổi...")
        tflite_model = converter.convert()
        print("[3/3] Lượng tử hóa thành công.")
        
        print(f"    -> Đang lưu mô hình TFLite tại: {dest_path}")
        with open(dest_path, 'wb') as f:
            f.write(tflite_model)
        
        print(f"\n*** THÀNH CÔNG! Đã tạo file: {tflite_name} (Kích thước: {os.path.getsize(dest_path)} bytes) ***")
    
    except Exception as e:
        print(f"\n!!! LỖI TOÀN BỘ KHI CHUYỂN ĐỔI: {e} !!!")
        print("--- Gợi ý: Hãy đảm bảo bạn đã CHẠY Ô 2 (định nghĩa class) trước khi chạy ô này. ---")

Thư mục làm việc hiện tại (BASE_DIR): e:\NCKH\dungcocach\Web_NGT\models
Đang tìm model Keras tại: e:\NCKH\dungcocach\Web_NGT\models\models\CNN_from_NPY_no_val.keras
Sẽ lưu TFLite tại: e:\NCKH\dungcocach\Web_NGT\models\models_tflite\CNN_from_NPY_no_val.tflite

!!! LỖI KIỂM TRA: Không tìm thấy file tại đường dẫn trên. !!!


In [ ]:
# --- CẤU HÌNH ---
# Lấy đường dẫn thư mục làm việc hiện tại (CWD)
BASE_DIR = os.getcwd() 
SOURCE_MODEL_DIR = os.path.join(BASE_DIR, "models")
DEST_TFLITE_DIR = os.path.join(BASE_DIR, "models_tflite")
MODEL_TEST_NAME = "CNN_from_NPY_no_val.keras"
# ------------------

model_path = os.path.join(SOURCE_MODEL_DIR, MODEL_TEST_NAME)
tflite_name = MODEL_TEST_NAME.replace(".keras", ".tflite")
dest_path = os.path.join(DEST_TFLITE_DIR, tflite_name)

os.makedirs(DEST_TFLITE_DIR, exist_ok=True)

print(f"Thư mục làm việc hiện tại (BASE_DIR): {BASE_DIR}")
print(f"Đang tìm model Keras tại: {model_path}")
print(f"Sẽ lưu TFLite tại: {dest_path}")

# KIỂM TRA QUAN TRỌNG: File có tồn tại không?
if not os.path.exists(model_path):
    print(f"\n!!! LỖI KIỂM TRA: Không tìm thấy file tại đường dẫn trên. !!!")
    print("Vui lòng đảm bảo file .keras của bạn nằm trong thư mục 'models' ở gốc.")
else:
    print("\nKiểm tra: Đã tìm thấy file model. Bắt đầu chuyển đổi...")
    try:
        # 1. Tải mô hình
        print("\n[1/4] Đang tải mô hình (load_model)...")
        # Chúng ta không cần 'custom_objects' vì các class đã được 'register' ở ô trên
        model = tf.keras.models.load_model(
            model_path, 
            compile=False  # Bỏ qua optimizer
        )
        print("[1/4] Tải mô hình thành công.")

        # 2. Khởi tạo bộ chuyển đổi
        print("[2/4] Đang khởi tạo bộ chuyển đổi TFLite...")
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        print("[2/4] Khởi tạo thành công.")

        # 3. Chuyển đổi
        print("[3/4] Đang tiến hành lượng tử hóa (convert)... Vui lòng chờ...")
        tflite_model = converter.convert()
        print("[3/4] Lượng tử hóa thành công.")

        # 4. Lưu file
        print(f"[4/4] Đang lưu mô hình TFLite tại: {dest_path}")
        with open(dest_path, 'wb') as f:
            f.write(tflite_model)
        print("[4/4] Lưu file thành công.")
        
        print(f"\n*** THÀNH CÔNG! Đã tạo file: {tflite_name} (Kích thước: {os.path.getsize(dest_path)} bytes) ***")

    except Exception as e:
        print(f"\n!!! LỖI TOÀN BỘ KHI CHUYỂN ĐỔI: {e} !!!")

In [ ]:
# CHUẨN BỊ DỮ LIỆU TỪ CÁC FILE .NPY

print("--- 3. Nạp Dữ liệu từ .npy ---")
try:
    X_train_all = np.load(X_TRAIN_PATH)
    y_train_all = np.load(Y_TRAIN_PATH)
    
    if os.path.exists(X_TEST_PATH) and os.path.exists(Y_TEST_PATH):
        X_test = np.load(X_TEST_PATH)
        y_test = np.load(Y_TEST_PATH)
        print("Đã tải thành công tập Train, Val và Test.")
    else:
        print("Cảnh báo: Không tìm thấy file X_test.npy hoặc y_test.npy.")
        X_test, y_test = None, None
        
    print(f"Shape X_train_all: {X_train_all.shape}")
    print(f"Shape y_train_all: {y_train_all.shape}")

    # Kiểm tra số lớp
    unique_labels = np.unique(y_train_all)
    if len(unique_labels) != NUM_CLASSES:
        print(f"Cảnh báo: Số lớp tìm thấy ({len(unique_labels)}) không khớp với NUM_CLASSES ({NUM_CLASSES}).")
        NUM_CLASSES = len(unique_labels)
        print(f"Đã cập nhật NUM_CLASSES thành {NUM_CLASSES}.")
    
    # Đảm bảo target_names_str khớp
    if len(target_names_str) != NUM_CLASSES:
        raise ValueError(f"Lỗi: 'target_names_str' có {len(target_names_str)} tên, nhưng NUM_CLASSES là {NUM_CLASSES}.")

    # Chuyển nhãn thành one-hot encoding
    y_train_all_ohe = to_categorical(y_train_all, num_classes=NUM_CLASSES)
    if y_test is not None:
        y_test_ohe = to_categorical(y_test, num_classes=NUM_CLASSES)

    # Nạp class weights nếu cần
    class_weights_dict = None
    if USE_CLASS_WEIGHTS:
        if os.path.exists(CLASS_WEIGHTS_PATH):
            class_weights_array = np.load(CLASS_WEIGHTS_PATH)
            class_weights_dict = dict(enumerate(class_weights_array))
            print("Đã tải và sẽ sử dụng class weights:")
            print(class_weights_dict)
        else:
            print("Cảnh báo: Đã bật USE_CLASS_WEIGHTS nhưng không tìm thấy file class_weights.npy.")
            USE_CLASS_WEIGHTS = False

except FileNotFoundError as e:
    print(f"Lỗi nghiêm trọng: Không tìm thấy file dữ liệu .npy! Vui lòng kiểm tra đường dẫn DATA_DIR.")
    print(e)
    # Dừng notebook ở đây nếu không có dữ liệu
    raise SystemExit("Dừng do thiếu dữ liệu.")
except Exception as e:
    print(f"Lỗi không xác định khi tải dữ liệu: {e}")
    raise SystemExit("Dừng do lỗi tải dữ liệu.")

In [ ]:
# HUẤN LUYỆN CROSS-VALIDATION (TỪ DỮ LIỆU .NPY)

print("\n--- 5. Bắt đầu Huấn luyện Cross-Validation ---")

fold_accuracies, fold_losses, fold_aucs, fold_f1s = [], [], [], []
# Sử dụng StratifiedKFold vì chúng ta đang làm việc với mảng NumPy
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_all, y_train_all)):
    fold_number = fold + 1
    print("-" * 60 + f"\nBắt đầu Fold {fold_number}/{N_SPLITS}\n" + "-" * 60)

    # --- 5A: Chuẩn bị dữ liệu cho Fold ---
    print("   - Chuẩn bị dữ liệu...")
    X_train_fold, X_val_fold = X_train_all[train_idx], X_train_all[val_idx]
    y_train_fold, y_val_fold = y_train_all_ohe[train_idx], y_train_all_ohe[val_idx]

    train_ds = tf.data.Dataset.from_tensor_slices((X_train_fold, y_train_fold))
    val_ds = tf.data.Dataset.from_tensor_slices((X_val_fold, y_val_fold))

    # Xóa các mảng NumPy lớn để tiết kiệm RAM
    del X_train_fold, X_val_fold, y_train_fold, y_val_fold
    gc.collect()

    # Áp dụng tiền xử lý, shuffle, batch, prefetch
    train_ds = train_ds.map(tf_preprocess_map, num_parallel_calls=AUTOTUNE)
    if USE_DATA_AUGMENTATION:
        train_ds = train_ds.map(augment_map, num_parallel_calls=AUTOTUNE) # Áp dụng Augmentation
        
    train_ds = train_ds.shuffle(SHUFFLE_BUFFER_SIZE).batch(GLOBAL_BATCH_SIZE).prefetch(AUTOTUNE)
    val_ds = val_ds.map(tf_preprocess_map, num_parallel_calls=AUTOTUNE).batch(GLOBAL_BATCH_SIZE).prefetch(AUTOTUNE)

    steps_per_epoch = len(train_idx) // GLOBAL_BATCH_SIZE
    validation_steps = len(val_idx) // GLOBAL_BATCH_SIZE
    print(f"   - Steps per epoch: {steps_per_epoch}, Validation steps: {validation_steps}")

    # --- 5B: Huấn luyện 2 Giai đoạn ---
    with strategy.scope():
        # SỬA ĐỔI: Sử dụng FinalModelCNN
        model = FinalModelCNN(
            input_shape_config=INPUT_SHAPE, 
            num_classes_config=NUM_CLASSES
        )
        
        if USE_FOCAL_LOSS:
            loss_function = tf.keras.losses.CategoricalFocalCrossentropy(from_logits=True, gamma=GAMMA, label_smoothing=LABEL_SMOOTHING_VALUE)
        else:
            loss_function = tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=LABEL_SMOOTHING_VALUE)
            
        # Giai đoạn 1: Huấn luyện Head (chỉ các lớp Dense)
        print("\n   --- Giai đoạn 1: Huấn luyện Head (Dense layers) ---")
        model.base_model.trainable = False
        optimizer_head = tf.keras.optimizers.AdamW(learning_rate=STAGE1_HEAD_LR, weight_decay=WEIGHT_DECAY, epsilon=1e-7)
        model.compile(optimizer=optimizer_head, loss=loss_function, metrics=['accuracy'], jit_compile=USE_XLA_COMPILATION)
        history_1a = model.fit(train_ds, validation_data=val_ds, epochs=STAGE1_HEAD_EPOCHS, 
                               steps_per_epoch=steps_per_epoch, validation_steps=validation_steps,
                               callbacks=[EarlyStopping(monitor='val_loss', patience=STAGE1_HEAD_PATIENCE, restore_best_weights=True)],
                               class_weight=class_weights_dict if USE_CLASS_WEIGHTS else None,
                               verbose=1)
        
        # Giai đoạn 2: Fine-tuning (toàn bộ model)
        print("\n   --- Giai đoạn 2: Fine-tuning ---")
        model.base_model.trainable = True
        
        if USE_COSINE_DECAY_RESTARTS:
            first_decay_steps = RESTART_CYCLE_1_EPOCHS * steps_per_epoch
            lr_scheduler = tf.keras.optimizers.schedules.CosineDecayRestarts(initial_learning_rate=STAGE1_FINETUNE_LR_INITIAL, first_decay_steps=first_decay_steps, t_mul=2.0, m_mul=0.9, alpha=0.1)
            optimizer_finetune = tf.keras.optimizers.AdamW(learning_rate=lr_scheduler, weight_decay=WEIGHT_DECAY, epsilon=1e-7)
            callbacks = [EarlyStopping(monitor='val_f1_macro', mode='max', patience=STAGE1_FINETUNE_PATIENCE, restore_best_weights=True, min_delta=MIN_DELTA, verbose=1), LearningRateLogger()]
        else:
            optimizer_finetune = tf.keras.optimizers.AdamW(learning_rate=STAGE1_FINETUNE_LR_INITIAL, weight_decay=WEIGHT_DECAY, epsilon=1e-7)
            callbacks = [EarlyStopping(monitor='val_f1_macro', mode='max', patience=STAGE1_FINETUNE_PATIENCE, restore_best_weights=True, verbose=1)]
        
        f1_macro = MacroF1Score(num_classes=NUM_CLASSES)
        model.compile(optimizer=optimizer_finetune, loss=loss_function, metrics=['accuracy', tf.keras.metrics.AUC(name='auc'), f1_macro], jit_compile=USE_XLA_COMPILATION)
        
        history_1b = model.fit(train_ds, validation_data=val_ds, epochs=STAGE1_FINETUNE_TOTAL_EPOCHS, 
                               steps_per_epoch=steps_per_epoch, validation_steps=validation_steps,
                               callbacks=callbacks,
                               class_weight=class_weights_dict if USE_CLASS_WEIGHTS else None,
                               verbose=1)
    
    # --- 5C: Lưu model, Vẽ biểu đồ, Đánh giá ---
    model_save_path = os.path.join(KAGGLE_OUTPUT_PATH, f'{MODEL_ID}_fold_{fold_number}.keras')
    model.save(model_save_path)
    print(f"\n   Đã lưu model cho Fold {fold_number} tại: {model_save_path}")

    print("\n   --- Vẽ biểu đồ huấn luyện ---")
    plot_training_history(history_1a, "Giai doan 1A - Head Training", fold_number, KAGGLE_OUTPUT_PATH)
    plot_training_history(history_1b, "Giai doan 1B - Fine-tuning", fold_number, KAGGLE_OUTPUT_PATH)

    print("\n   --- Đánh giá trên tập Validation ---")
    val_results = model.evaluate(val_ds, verbose=0, return_dict=True)
    loss, accuracy, auc, f1 = val_results.get('loss', 0), val_results.get('accuracy', 0), val_results.get('auc', 0), val_results.get('f1_macro', 0)
    print(f"   Fold {fold_number} - Validation Loss: {loss:.4f}, Accuracy: {accuracy:.4f}, AUC: {auc:.4f}, F1-Macro: {f1:.4f}")
    
    fold_accuracies.append(accuracy); fold_losses.append(loss); fold_aucs.append(auc); fold_f1s.append(f1)
    
    print("\n   " + "=" * 50 + "\n   Kết quả Cross-Validation Tạm thời:\n" 
          + f"     - Accuracy trung bình: {np.mean(fold_accuracies):.4f} +/- {np.std(fold_accuracies):.4f}\n"
          + f"     - Loss trung bình: {np.mean(fold_losses):.4f} +/- {np.std(fold_losses):.4f}\n"
          + f"     - AUC trung bình: {np.mean(fold_aucs):.4f} +/- {np.std(fold_aucs):.4f}\n"
          + f"     - F1-Macro trung bình: {np.mean(fold_f1s):.4f} +/- {np.std(fold_f1s):.4f}\n"
          + "   " + "=" * 50)
          
    # --- 5D: Dọn dẹp bộ nhớ ---
    print("\n   --- Dọn dẹp bộ nhớ ---")
    try:
        del model, train_ds, val_ds
        tf.keras.backend.clear_session()
        gc.collect()
        print("   Đã dọn dẹp bộ nhớ thành công.")
    except NameError as e:
        print(f"   Một số biến có thể đã được dọn dẹp, bỏ qua lỗi: {e}")

# --- IN KẾT QUẢ TỔNG KẾT CUỐI CÙNG ---\
print("\n\n" + "=" * 60 + "\nKẾT QUẢ CROSS-VALIDATION CUỐI CÙNG:\n" 
      + f"  - Validation Accuracy trung bình: {np.mean(fold_accuracies):.4f} +/- {np.std(fold_accuracies):.4f}\n"
      + f"  - Validation Loss trung bình: {np.mean(fold_losses):.4f} +/- {np.std(fold_losses):.4f}\n"
      + f"  - Validation AUC trung bình: {np.mean(fold_aucs):.4f} +/- {np.std(fold_aucs):.4f}\n"
      + f"  - Validation F1-Macro trung bình: {np.mean(fold_f1s):.4f} +/- {np.std(fold_f1s):.4f}\n"
      + "=" * 60)

In [ ]:
# ĐÁNH GIÁ CUỐI CÙNG TRÊN TẬP TEST (HOLD-OUT)

if X_test is not None and y_test is not None:
    print("\n--- 6. Bắt đầu Đánh giá cuối cùng trên tập Test (Hold-out) ---")
    
    # Biến target_names_str đã được định nghĩa ở Cell 3
    if len(target_names_str) != NUM_CLASSES:
        raise ValueError("Lỗi: 'target_names_str' và 'NUM_CLASSES' không khớp.")
    
    # --- 6A: Tạo Test Dataset ---
    print("   - Chuẩn bị Test Dataset...")
    test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test_ohe))
    test_ds = test_ds.map(tf_preprocess_map, num_parallel_calls=AUTOTUNE) \
                       .batch(GLOBAL_BATCH_SIZE) \
                       .prefetch(AUTOTUNE)

    # --- 6B: Lấy dự đoán từ 5 model ---
    all_fold_preds = []
    print(f"   - Lấy dự đoán từ {N_SPLITS} models...")
    for fold_number in range(1, N_SPLITS + 1):
        model_path = os.path.join(KAGGLE_OUTPUT_PATH, f'{MODEL_ID}_fold_{fold_number}.keras')
        if os.path.exists(model_path):
            print(f"     - Đang tải model Fold {fold_number}...")
            with strategy.scope():
                # SỬA LỖI: Đổi 'FinalModelCRNN' thành 'FinalModelCNN'
                model = tf.keras.models.load_model(model_path, custom_objects={'FinalModelCNN': FinalModelCNN, 'MacroF1Score': MacroF1Score})
            
            print(f"     - Đang dự đoán với model Fold {fold_number}...")
            fold_preds = model.predict(test_ds, verbose=1)
            all_fold_preds.append(fold_preds)
            
            del model
            tf.keras.backend.clear_session()
            gc.collect()
        else:
            print(f"     - Cảnh báo: Không tìm thấy model Fold {fold_number} tại '{model_path}'.")

    # --- 6C: Tính trung bình dự đoán (Ensemble) ---
    if all_fold_preds:
        print("\n   - Tính trung bình dự đoán...")
        avg_preds = np.mean(all_fold_preds, axis=0)
        y_pred_probs_final = tf.nn.softmax(avg_preds).numpy() # Áp dụng softmax cho logits trung bình
        y_pred_final = np.argmax(y_pred_probs_final, axis=1)
        y_true_final = y_test # Sử dụng nhãn gốc (integer)
        
        # --- 6D: Tính toán và In các chỉ số cuối cùng ---
        print("\n   - Kết quả đánh giá cuối cùng trên tập Test:")
        
        # Classification Report
        print("\nClassification Report:\n", classification_report(y_true_final, y_pred_final, target_names=target_names_str))
        
        # Confusion Matrix
        cm = confusion_matrix(y_true_final, y_pred_final)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=target_names_str, yticklabels=target_names_str)
        plt.xlabel('Predicted Label')
        plt.ylabel('True Label')
        plt.title('Confusion Matrix on Test Set')
        cm_path = os.path.join(KAGGLE_OUTPUT_PATH, f'{MODEL_ID}_confusion_matrix.png')
        plt.savefig(cm_path)
        print(f"   Đã lưu Confusion Matrix tại: {cm_path}")
        plt.show()

        # Tính AUC đa lớp (One-vs-Rest)
        y_test_ohe_for_auc = y_test_ohe
        y_pred_probs_for_auc = y_pred_probs_final
        
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for i in range(NUM_CLASSES):
            fpr[i], tpr[i], _ = roc_curve(y_test_ohe_for_auc[:, i], y_pred_probs_for_auc[:, i])
            roc_auc[i] = sklearn_auc(fpr[i], tpr[i])

        # (Phần tính Micro và Macro AUC giữ nguyên)
        fpr["micro"], tpr["micro"], _ = roc_curve(y_test_ohe_for_auc.ravel(), y_pred_probs_for_auc.ravel())
        roc_auc["micro"] = sklearn_auc(fpr["micro"], tpr["micro"])
        all_fpr = np.unique(np.concatenate([fpr[i] for i in range(NUM_CLASSES)]))
        mean_tpr = np.zeros_like(all_fpr)
        for i in range(NUM_CLASSES):
            mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
        mean_tpr /= NUM_CLASSES
        fpr["macro"] = all_fpr
        tpr["macro"] = mean_tpr
        roc_auc["macro"] = sklearn_auc(fpr["macro"], tpr["macro"])

        # Vẽ ROC Curves
        plt.figure(figsize=(10, 8))
        plt.plot(fpr["micro"], tpr["micro"], label=f'Micro-average ROC (AUC = {roc_auc["micro"]:.2f})', color='deeppink', linestyle=':', linewidth=4)
        plt.plot(fpr["macro"], tpr["macro"], label=f'Macro-average ROC (AUC = {roc_auc["macro"]:.2f})', color='navy', linestyle=':', linewidth=4)
        colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'green'])
        for i, color in zip(range(NUM_CLASSES), colors):
            plt.plot(fpr[i], tpr[i], color=color, lw=2, label=f'ROC curve of class {target_names_str[i]} (AUC = {roc_auc[i]:.2f})')

        plt.plot([0, 1], [0, 1], 'k--', lw=2)
        plt.xlim([0.0, 1.0]); plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic (ROC) on Test Set')
        plt.legend(loc="lower right")
        roc_path = os.path.join(KAGGLE_OUTPUT_PATH, f'{MODEL_ID}_roc_curve.png')
        plt.savefig(roc_path)
        print(f"   Đã lưu ROC Curve tại: {roc_path}")
        plt.show()
    else:
        print("\n   Không thể thực hiện đánh giá cuối cùng do thiếu kết quả từ các fold.")
else:
     print("\n--- Bỏ qua Đánh giá cuối cùng trên tập Test do thiếu dữ liệu Test (X_test hoặc y_test là None) ---")

print("\n=== QUY TRÌNH HOÀN TẤT ====")

NameError: name 'X_test' is not defined

In [ ]:
# PHÂN TÍCH GRAD-CAM TRÊN TẬP TEST
# Logic Grad-CAM này vẫn hoạt động cho CNN vì nó tìm 'top_conv' bên trong base_model

print("--- Chuẩn bị dữ liệu Test cho việc phân tích Grad-CAM ---\n")
if 'test_ds' in locals(): # Kiểm tra xem test_ds đã được tạo ở ô trước chưa
    
    @tf.function
    def get_grad_cam_batched(model, img_batch):
        """
        Phiên bản Grad-CAM linh hoạt, hoạt động với các model lồng nhau.
        """
        # Tìm lớp Conv cuối cùng trong base_model
        last_conv_layer = model.base_model.get_layer("top_conv")
        
        # Tạo grad_model
        grad_model = tf.keras.models.Model(
            [model.inputs], [last_conv_layer.output, model.output]
        )
        
        with tf.GradientTape() as tape:
            last_conv_layer_output_value, preds = grad_model(img_batch)
            pred_indices = tf.argmax(preds, axis=1)
            class_channels = tf.gather(preds, pred_indices, axis=1, batch_dims=1)

        grads = tape.gradient(class_channels, last_conv_layer_output_value)
        pooled_grads = tf.reduce_mean(grads, axis=(1, 2))
        
        heatmap_batch = tf.einsum('bhwc,bc->bhw', last_conv_layer_output_value, pooled_grads)
        heatmap_batch = tf.maximum(heatmap_batch, 0)
        
        max_vals = tf.reduce_max(heatmap_batch, axis=(1, 2), keepdims=True)
        heatmap_batch = heatmap_batch / (max_vals + tf.keras.backend.epsilon())
        
        return heatmap_batch, preds

    # Hàm run_grad_cam_analysis_final
    def run_grad_cam_analysis_final(model, model_name, output_base_path, test_dataset):
        print(f"\n--- Bắt đầu phân tích cho mô hình: {model_name} ---")
        
        # SỬA LỖI: Sử dụng 'target_names_str' thay vì 'target_names'
        results_by_class = { name: {'correct_heatmaps': [], 'correct_confidences': [], 'correct_images': [],
                                    'incorrect_heatmaps': [], 'incorrect_confidences': [], 'incorrect_images': []}
                            for name in target_names_str }

        print("  - Xử lý các batch trên dataset...")
        for images_batch, labels_batch in tqdm(test_dataset, desc=f"Analyzing {model_name}"):
            heatmap_batch, preds_batch = get_grad_cam_batched(model, images_batch)
            y_pred_probs_batch = tf.nn.softmax(preds_batch).numpy()
            y_pred_batch = np.argmax(y_pred_probs_batch, axis=1)
            y_true_batch = np.argmax(labels_batch.numpy(), axis=1)

            for i in range(images_batch.shape[0]):
                y_pred, y_true = y_pred_batch[i], y_true_batch[i]
                # SỬA LỖI: Sử dụng 'target_names_str'
                true_class_name = target_names_str[y_true] 
                
                if y_pred == y_true:
                    results_by_class[true_class_name]['correct_heatmaps'].append(heatmap_batch[i].numpy())
                    results_by_class[true_class_name]['correct_confidences'].append(y_pred_probs_batch[i, y_pred])
                    results_by_class[true_class_name]['correct_images'].append(images_batch[i].numpy())
                else:
                    results_by_class[true_class_name]['incorrect_heatmaps'].append(heatmap_batch[i].numpy())
                    results_by_class[true_class_name]['incorrect_confidences'].append(y_pred_probs_batch[i, y_pred])
                    results_by_class[true_class_name]['incorrect_images'].append(images_batch[i].numpy())
        
        # (Bạn có thể thêm code ở đây để lưu các heatmap đã thu thập được)
        print(f"  - Phân tích cho {model_name} hoàn tất.")

    
    # --- VÒNG LẶP CHÍNH ĐỂ PHÂN TÍCH 5 FOLDS ---
    grad_cam_main_path = os.path.join(KAGGLE_OUTPUT_PATH, "grad_cam_detailed_analysis")
    os.makedirs(grad_cam_main_path, exist_ok=True)
    
    for fold_number in range(1, N_SPLITS + 1):
        print(f"\n---> Bắt đầu phân tích Grad-CAM cho Fold {fold_number}/{N_SPLITS}...")
        model_path = os.path.join(KAGGLE_OUTPUT_PATH, f'{MODEL_ID}_fold_{fold_number}.keras')
        if not os.path.exists(model_path):
            print(f"Bỏ qua Fold {fold_number}, không tìm thấy file.")
            continue
        
        try:
            with strategy.scope():
                # SỬA LỖI: Đổi 'FinalModel' thành 'FinalModelCNN'
                model = tf.keras.models.load_model(model_path, custom_objects={'FinalModelCNN': FinalModelCNN, 'MacroF1Score': MacroF1Score})
            
            model_name = f"fold_{fold_number}"
            fold_output_path = os.path.join(grad_cam_main_path, model_name)
            os.makedirs(fold_output_path, exist_ok=True)
            
            # Gọi hàm phân tích với test_ds (đã được batch)
            run_grad_cam_analysis_final(model, model_name, fold_output_path, test_ds)
        
        except Exception as e:
            print(f"!!! Lỗi khi phân tích Grad-CAM cho Fold {fold_number}: {e}")
            
    print("\n--- Toàn bộ quá trình phân tích Grad-CAM đã hoàn tất ---")
else:
    print("Lỗi: Không tìm thấy 'test_ds'. Vui lòng chạy ô đánh giá (Cell 7) trước.")

In [ ]:
# CHUYỂN ĐỔI SANG TFLITE

print("--- Bắt đầu quy trình chuyển đổi 5-Fold sang TFLite ---")
if 'fold_f1s' in locals() and len(fold_f1s) == N_SPLITS:
    
    # Dùng X_train_all để tạo representative dataset
    
    for fold_number in range(1, N_SPLITS + 1):
        print("=" * 60)
        print(f"--- Bắt đầu chuyển đổi cho Fold {fold_number} ---")
        
        MODEL_PATH = os.path.join(KAGGLE_OUTPUT_PATH, f'{MODEL_ID}_fold_{fold_number}.keras')
        TFLITE_MODEL_PATH = os.path.join(KAGGLE_OUTPUT_PATH, f'model_fold_{fold_number}_quantized.tflite')

        if not os.path.exists(MODEL_PATH):
            print(f"Lỗi: Không tìm thấy file model tại '{MODEL_PATH}'. Bỏ qua fold này.")
            continue

        print("Đang tải lại model .keras...")
        with strategy.scope():
            # SỬA LỖI: Đổi 'FinalModel' thành 'FinalModelCNN'
            model_to_convert = tf.keras.models.load_model(MODEL_PATH, custom_objects={'FinalModelCNN': FinalModelCNN, 'MacroF1Score': MacroF1Score})

        print(f"Đang tạo representative dataset...")
        def representative_data_gen():
            # Lấy 150 mẫu ngẫu nhiên từ TẬP TRAIN GỐC để hiệu chỉnh
            for i in np.random.choice(len(X_train_all), 150, replace=False):
                img = X_train_all[i]
                # Sử dụng hàm tiền xử lý đã được cập nhật
                img_processed = preprocess_npy_image(img) 
                yield [tf.expand_dims(img_processed, axis=0)]

        print(f"Đang chuyển đổi mô hình của Fold {fold_number}...")
        converter = tf.lite.TFLiteConverter.from_keras_model(model_to_convert)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.representative_dataset = representative_data_gen
        converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
        converter.inference_input_type = tf.float32
        converter.inference_output_type = tf.float32

        tflite_quant_model = converter.convert()

        with open(TFLITE_MODEL_PATH, 'wb') as f:
            f.write(tflite_quant_model)
        
        print(f"Đã lưu thành công model TFLite cho Fold {fold_number} tại: {TFLITE_MODEL_PATH}")
        print(f"Kích thước file: {len(tflite_quant_model) / (1024 * 1024):.2f} MB")
        
        del model_to_convert
        gc.collect()
        tf.keras.backend.clear_session()

    print("=" * 60)
    print("\nHoàn tất chuyển đổi cho cả 5 mô hình!")
else:
    print("Lỗi: Không tìm thấy kết quả của 5 fold. Vui lòng chạy ô huấn luyện (Cell 6) trước.")